# Topic modeling paso a paso: los temas de la pandemia

Esta notebook muestra las etapas de una experiencia de modelización de tópicos con un corpus de tweets sobre la pandemia de coronavirus. Aquí explicamos cómo:

- preprocesar los datos
- entrenar modelos con LDA
- generar visualizaciones de los resultados

Autora: Nidia Hernández, CAICYT-CONICET, nidiahernandez@conicet.gov.ar

## Requerimientos

Primero, nos aseguramos de instalar las librerías necesarias y otros requerimientos.

In [1]:
#! pip install requirements.txt
! python -m nltk.downloader stopwords

/opt/conda/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Importamos las librerías y las funciones que vamos a usar para el procesamiento:

In [2]:
from os.path import isfile
from os import makedirs
import re

from detectar_topicos import * # Importa las funciones del script detectar_topicos.py

from nltk.tokenize import RegexpTokenizer

from tqdm import tqdm

import gensim
from gensim import corpora
from gensim.models import Phrases
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pandas as pd
import matplotlib.pyplot as plt
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Carga del corpus

Usamos https://covid.dh.miami.edu/get/ para obtener una colección de tweets por fecha. Los tweets son descargados en formato txt, un tweet por línea. 

Aquí vamos a trabajar con los tweets sobre el covid19 del 6 de noviembre de 2020 en Argentina, el día en finalizó el aislamiento social preventivo y obligatorio en la Ciudad de Buenos Aires.

In [3]:
corpus_path = 'dhcovid_2020-11-06_es_ar.txt'
corpus_label = corpus_path.replace('.txt', '')

with open(corpus_path, 'r') as fi:
    tweets = fi.read()
    print('Tweets totales: ', len(tweets.split('\n')))
    tweets = set(tweets.split('\n') ) # elimina duplicados
    tweets = list(list(tweets))

print(f'Tweets sin duplicados: {len(tweets)}')

Tweets totales:  2177
Tweets sin duplicados: 2142


Veamos los diez primeros tweets de esta colección:

In [4]:
tweets[:15]

['la defensa covid19 76 casos 2 muertos URL',
 'en vivo reporte desde el ministerio de salud nuevo URL',
 'casado arranca su segunda temporada en italia en medio de los rebrotes de covid19 URL',
 'spoiler si trump pierde las elecciones el covid19 desaparece si trump gana la 2da ola va a ser aun peor que la primera cada dia mas convencido de que es un virus intencional',
 'por un lado tengo a mi viejo que le sube un poco la fiebre y ya llora por su muerte y por el otro tengo al obsesivo por la limpieza llorando por q siente el covid19 en sus manos y recien estamos en el primer dia de aislamiento q hice en mi otra vida para merecer 2 trolos llorones',
 'otra noche otro servicio otro accidente otra vez cambiandome con la proteccion para el covid19 otra vez acostandome a las 4 am',
 'apocrifo 127 pizza con covid19 mientras estaba cenando en el local de @user del cerro de las rosas llego la policia para avisar que un empleado habia dado positivo de covid19 #yomequedoencasa #coronavirus URL 

Hay muchos tokens irrelevantes para nuestro objetivo de detección de tópicos: 'URL', '@user', números, palabras vacías de contenido informativo, etc.

## Preprocessing

Es habitual mejorar la calidad de los datos de entrada del topic modeling realizando diversos tipos de preprocesamiento. Los más rápidos y sencillos son eliminar tokens poco pertinentes. Otros pueden ser computacionalmente más costosos pero mejoran notablemente la legibilidad de los resultados. En esta experiencia, realizaremos un pretratamiento standard: filtrar tokens, generar bigramas y marcar Named Entities.

Creamos una etiqueta para identificar los modelos de acuerdo al pretratamiento que vamos a realizar:

In [5]:
model_label = '2gram_ner_LDA'

En primer lugar, eliminamos las transliteraciones de emojis usando la función `remove_emojis`:

In [6]:
tweets_noemojis = [remove_emojis(tweet).strip() for tweet in tweets]
tweets_noemojis = [tweet for tweet in tweets_noemojis if tweet]
tweets_noemojis = [tweet.split() for tweet in tweets_noemojis]

Luego, eliminamos las palabras gramaticales ('la', 'que', 'de', etc) ya que son muy frecuentes pero no aportan información temática significativa. En el ámbito del _text processing_ estas palabras son conocidas como _stopwords_. Cargamos una lista genérica de _stopwords_ del español y le añadimos tokens particulares de nuestro corpus:

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')
stop_words_extra = ['@user', '#covid19', '#covid','#coronavirus','URL','xq','pq', 'q', 'd', 'x', 'e', 'k', 'l', 're','ja', 'jaja'
                    'si', 'mas','da','dia', 'hoy', 'año', 'aca', 'ahi', 'aqui', 'vez', 'tras', 'traves', 'bueno']
stop_words = stop_words+stop_words_extra

print(f"[{corpus_label}-{model_label}] Filtrando stopwords")
tweets_filtrados = [[token for token in texto if token not in stop_words] for texto in tweets_noemojis]

[dhcovid_2020-11-06_es_ar-2gram_ner_LDA] Filtrando stopwords


Nos interesa que conservar expresiones como "nuevo caso" o "vacuna rusa" porque ayudan notablemente a la lectura de resultados frente a las mismas palabras por separado. Esto lo logramos generando los bigramas de los tweets y conservando los que aparecen al menos 15 veces en la colección:

In [8]:
print(f"[{corpus_label}-{model_label}] Generando bigramas")
bigram = Phrases(tweets, min_count=15)

tweets_bigrams = tweets_filtrados.copy()
for idx in tqdm(range(len(tweets_filtrados))):
    for token in bigram[tweets_filtrados[idx]]:
        if '_' in token:
            tweets_bigrams[idx].append(token)

[dhcovid_2020-11-06_es_ar-2gram_ner_LDA] Generando bigramas


100%|██████████| 2142/2142 [00:00<00:00, 59957.02it/s]


Filtramos los números después de generar los bigramas para conservar expresiones como '24_horas'.

In [9]:
tweets_bigrams_filt = [[token for token in texto if not token.isnumeric()] for texto in tweets_bigrams]

Otro procesamiento que permite lograr resultados más claros para la lectura humana es la identificación de Named Entities. Esta técnica nos permite detectar expresiones como 'hospital ramos mejia' para marcarlas así 'hospital_ramos_mejia'.

Previamente, usamos Spacy para detectar automáticamente las Named Entities de esta colección y las volcamos en una lista que revisamos manualmente para eliminar falsos positivos.

In [10]:
# import spacy
# spacy_nlp = spacy.load('es_core_news_lg')
# tweets_spacy = [spacy_nlp(' '.join(tweet), disable=["tagger", "parser"]) for tweet in tqdm(tweets_bigrams_filt)]

# with open(f'{corpus_label}_NE.lst', 'w') as fi:
#     for tweet in tweets_spacy:
#         for entity in tweet.ents:
#             entity_words = str(entity).split()
#             if len(entity_words) > 1:
#                 fi.write(f'{" ".join(entity_words)}\n')

In [11]:
print(f"[{corpus_label}-{model_label}] Identificando Named Entities")
with open(f'dhcovid_es_ar_NE.lst', 'r') as fi: # lista de NE revisada manualmente
    entidades_curadas = fi.read().split('\n')

tweets_ner = []
for texto in tqdm(tweets_bigrams_filt):
    texto = ' '.join(texto)
    for entity in entidades_curadas:
        entity_merged = '_'.join(entity.split())
        texto = texto.replace(entity, entity_merged)
    tweets_ner.append(texto.split())
    

  7%|▋         | 140/2142 [00:00<00:01, 1392.09it/s]

[dhcovid_2020-11-06_es_ar-2gram_ner_LDA] Identificando Named Entities


100%|██████████| 2142/2142 [00:01<00:00, 1382.45it/s]


Todas estas manipulaciones del corpus pueden ser costosas, por eso conviene guardar una copia del resultado:

In [12]:
processed_tweets_path = corpus_path.replace(".txt", ".processed-tweets.json")

print(f"[{corpus_label}] Guardando copia de tweets procesados")
dump_processed_tweets_as_json(tweets_ner, processed_tweets_path)

[dhcovid_2020-11-06_es_ar] Guardando copia de tweets procesados


## Entrenamiento

Una vez que los datos de entrada fueron adaptados, podemos proceder al aprendizaje no supervisado de tópicos. Como los tópicos emergen automáticamente de los datos, no podemos saber de antemano cuántos serán. El parámetro `topic_numbers_to_try` permite configurar el rango de tópicos a entrenar.

En este paso, también podemos refinar el corpus de entranda. El parámetro `filter_extremes` permite excluir palabras de frecuencia muy baja o demasiado alta (ver la función `make_dictionary_and_matrix` en `detectar_topicos.py`).

⚠️ El aprendizaje puede llevar muchas horas si el corpus es grande.

In [13]:
print(f"[{corpus_label}-{model_label}] Unsupervised learning")
models_dir='./dhcovid_tweets_models'

models = train_several_LDA_models(
    documentos=tweets_ner,
    topic_numbers_to_try=range(3, 51),
    corpus_label=corpus_label,
    model_label=model_label,
    models_dir=models_dir,
    overwrite=True, # False: carga modelos previamente entrenados si existen. True: los reescribe.
    filter_extremes=True,
)

[dhcovid_2020-11-06_es_ar-2gram_ner_LDA] Unsupervised learning
Training LDA model with 3 topics


KeyboardInterrupt: 

¿Cómo saber cuál es el modelo con el número óptimo de tópicos?

## Evaluación automática

En el paso anterior, entrenamos modelos para un amplio rango de tópicos. ¿Cómo saber cuál es el que mejor representa las temáticas que se tratan en nuestro corpus de tweets de Argentina del 6 de noviembre de 2020 sobre el covid19?

Una manera de encontrar _automáticamente_ el modelo con el mejor número de tópicos es usar un score de coherencia. Existen varias medidas que permiten evaluar la coherencia de los modelos generados. En esta notebook vamos a usar _cv_.

In [ ]:
scores = calculate_topic_coherence(models, tweets_ner, measures=["c_v"], filter_extremes=True)
scores.to_csv(f'{models_dir}/{corpus_label}-{model_label}-coherence.csv')
plot_cv(scores, corpus_label, model_label, models_dir)

ntopics_with_top_cv_score = scores.set_index("ntopics").c_v.idxmax()
cv_score = round(scores.c_v.max(), 2)

print(f"El modelo más coherente tiene {ntopics_with_top_cv_score} tópicos y recibió un score de c_v de {cv_score}")

El score automático nos permite descartar rápidamente varios modelos, pero es necesario encontrar un compromiso entre el valor del score y un número de tópicos razonable para la evaluación cualitativa.

## Análisis de resultados

Cada modelo tiene una cantidad de tópicos posibles. A su vez, cada tópico está integrado por las palabras más probables para ese tópico y la probabilidad asociada. Para visualizar toda esta información, construimos una tabla con los tópicos, las palabras y las probabilidades para el modelo que recibió el mejor scoring.

In [ ]:
best_model = models[ntopics_with_top_cv_score]
tabla = make_table_all_topics(best_model, model_label, corpus_label)
tabla.to_csv(f'{models_dir}/{corpus_label}-{model_label}-topics.csv')
tabla.head(10)

Ahora, para cada tópico, visualizamos las palabras que lo componen:

In [ ]:
topic_total = tabla.topic_id.unique()
for topic_id in topic_total:
    topic = tabla.query("topic_id == @topic_id")
    print(f'Tópico {topic_id}: ', " | ".join(topic.word))

Podemos generar una visualización interactiva con más información donde podemos observar fácilmente cantidad de tópicos, distancia, peso y palabras de cada tópico:

In [ ]:
plot_LDA_topics(best_model, tweets_ner, output_path=f'{models_dir}/{corpus_label}-{model_label}.html', filter_extremes=True)